In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

import h5py
import numpy as np
import seaborn as sns

from astropy import constants as const
from astropy import units as u


m_p = const.m_p.cgs.to(u.M_sun).value

cm = u.cm.to(u.pc)
g = u.g.to(u.M_sun)

# Create cluster initial conditions

In [ ]:
icf = h5py.File("single_ics.hdf5", "w")

In [ ]:
header_group = icf.create_group("Header")
particles_group = icf.create_group("PartType0")


In [ ]:
box_size = 200
n_part_per_side = 200

n_part_gas = n_part_per_side**3
n_part = np.array([n_part_gas, 0, 0, 0, 0, 0])

In [ ]:
header_group.attrs["NumPart_ThisFile"] = n_part
header_group.attrs["NumPart_Total"]    = n_part
header_group.attrs["NumPart_Total_HighWord"] = np.zeros(6)

header_group.attrs["MassTable"] = np.zeros(6)

header_group.attrs["Time"] = 0
header_group.attrs["Redshift"] = 0
header_group.attrs["BoxSize"] = box_size
header_group.attrs["NumFilesPerSnapshot"] = 1
header_group.attrs["Omega0"] = 0.0
header_group.attrs["OmegaLambda"] = 0.0
header_group.attrs["HubbleParam"] = 1.0
header_group.attrs["Flag_Sfr"] = 0
header_group.attrs["Flag_Cooling"] = 0
header_group.attrs["Flag_StellarAge"] = 0
header_group.attrs["Flag_Metals"] = 0
header_group.attrs["Flag_Feedback"] = 0
header_group.attrs["Flag_DoublePrecision"] = 0
header_group.attrs["Flag_IC_Info"] = 0




In [ ]:
particles_group.create_dataset("Coordinates",     (n_part_gas, 3), dtype="<f4")
particles_group.create_dataset("Density",         (n_part_gas,  ), dtype="<f4")
particles_group.create_dataset("ParticleIDs",     (n_part_gas,  ), dtype="<u4")
particles_group.create_dataset("Velocities",      (n_part_gas, 3), dtype="<f4")
particles_group.create_dataset("InternalEnergy",  (n_part_gas,  ), dtype="<f4")
particles_group.create_dataset("Masses",          (n_part_gas,  ), dtype="<f4")

In [ ]:
xx, yy, zz = np.meshgrid(np.linspace(0,box_size,num=n_part_per_side+1)[1:], 
                         np.linspace(0,box_size,num=n_part_per_side+1)[1:], 
                         np.linspace(0,box_size,num=n_part_per_side+1)[1:])
dx = xx[0,1,0] - xx[0,0,0]

In [ ]:
particles_group["Coordinates"][:,0] = xx.flatten()
particles_group["Coordinates"][:,1] = yy.flatten()
particles_group["Coordinates"][:,2] = zz.flatten()

particles_group["Coordinates"][:,0] *= 1 + (np.random.random(size=n_part_gas)-.5)*1e-3
particles_group["Coordinates"][:,1] *= 1 + (np.random.random(size=n_part_gas)-.5)*1e-3
particles_group["Coordinates"][:,2] *= 1 + (np.random.random(size=n_part_gas)-.5)*1e-3

particles_group["Coordinates"][:] = np.array(particles_group["Coordinates"]) % 500

density = 1.33 * m_p / cm**3
particles_group["Density"][:] = density

particles_group["ParticleIDs"][:] = np.arange(1, n_part_gas+1, dtype="<u4")

particles_group["Velocities"][:] = 0
particles_group["InternalEnergy"][:] = 0
particles_group["Masses"][:] = dx**3 * density


In [ ]:
list(particles_group.keys())

In [ ]:
icf.close()